**Preprocess the data**

In [1]:
import pandas as pd
from datetime import datetime
import time

# data1030-student-f19/data1030_project/data/News_Final.csv
df = pd.read_csv('../data/News_Final.csv')
print(df.shape)

(93239, 11)


In [2]:
df = df.drop(columns=['Title', 'Headline', 'Source'])
print(df.head(2))

    IDLink    Topic          PublishDate  SentimentTitle  SentimentHeadline  \
0  99248.0    obama  2002-04-02 00:00:00        0.000000          -0.053300   
1  10423.0  economy  2008-09-20 00:00:00        0.208333          -0.156386   

   Facebook  GooglePlus  LinkedIn  
0        -1          -1        -1  
1        -1          -1        -1  


In [3]:
from sklearn.preprocessing import MinMaxScaler

# change to unix time
df_time = df['PublishDate']
x = df_time.values
unixTime = [time.mktime(pd.Timestamp(y).timetuple()) for y in x]
df['PublishDate'] = unixTime

# minmaxscaler
df_time = df[['PublishDate']]

scaler = MinMaxScaler()
scaler.fit(df_time)
df_time_maxmin = scaler.transform(df_time)
df['PublishDate'] = df_time_maxmin

In [4]:
df_plat = df[['Facebook','GooglePlus','LinkedIn']]

scaler = MinMaxScaler()
scaler.fit(df_plat)
df_plat_maxmin = scaler.transform(df_plat)

platName = [x+str('_pro') for x in ['Facebook','GooglePlus','LinkedIn']]

for i in range(len(platName)):
    df[platName[i]] = df_plat_maxmin[:,i]

In [5]:
from sklearn.preprocessing import OneHotEncoder
df_topic = df[['Topic']]
#print(df_topic)

enc = OneHotEncoder(sparse=False) 
enc.fit(df_topic)
#print('categories:',enc.categories_)

enc_label = enc.get_feature_names()
print('feature names:',enc_label)
df_topic_ohe = enc.transform(df_topic)
print(df_topic_ohe)

print(df_topic_ohe[:,0])

feature names: ['x0_economy' 'x0_microsoft' 'x0_obama' 'x0_palestine']
[[0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 ...
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 1. 0. 0.]]
[0. 1. 1. ... 0. 0. 0.]


In [6]:
for i in range(len(enc_label)):
    df[enc_label[i]] = df_topic_ohe[:,i]

In [7]:
# target var.
# if there are two or more -1:
# target = 0
# else, target = the platform with the highest popularity:
# F = 1; G = 2; L = 3
# if the news has been posted on all three platform but all three platforms has zero popularity:
# target = 4
# all the other samples, f = g > l etc.:
# target = 5

t = []
for i in range(df.shape[0]):
    f = df['Facebook'].values[i]
    g = df['GooglePlus'].values[i]
    l = df['LinkedIn'].values[i]
    if sum([(f == -1), (g == -1), (l == -1)]) > 1:
        t.append(0) # 5745
    elif f > g and f > l:
        t.append(1) # 53721
    elif g > f and g > l:
        t.append(2) # 2297
    elif l > f and l > g:
        t.append(3) # 12105
    elif f == 0 and f == g and g == l:
        t.append(4) # 14875
    else:
        t.append(5) # 4496
df['BestPlat'] = t

In [9]:
df.shape

(93239, 16)

In [17]:
export_csv = df.to_csv ('../data/News_pro.csv', index = None, header=True) 